In [ ]:
! pip3 install onnxoptimizer==0.2.6
! pip list | grep onnx

In [130]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from onnx.helper import make_graph, make_model, set_model_props
from onnx import helper, TensorProto
from qonnx.core.datatype import DataType
import numpy as np
import onnx

def get_init(model, tensor_name):
    for init in model.graph.initializer:
        if tensor_name == init.name:
            init_val = init.raw_data
            model.graph.initializer.remove(init)
            break
    for inp in model.graph.input:
        if tensor_name in inp.name:
            model.graph.input.remove(inp)
    
    return np.frombuffer(init_val, dtype = np.int64)

def add_attr(model, op_type):
    new_node = []
    for ind,n in enumerate(model.graph.node):
        new_attr = {}
        if op_type in n.name:        
            if len(n.input) > 1:
                new_attr["starts"] = get_init(model,n.input[1])
                new_attr["ends"] = get_init(model,n.input[2])
                new_attr["axes"] = get_init(model,n.input[3])
            if len(n.input[1:]) >3:
                new_attr["steps"] = get_init(model,n.input[4])
            if not new_attr == {}:
                new_node = make_node(model,new_attr,op_type)
                model.graph.node.remove(n)
                model.graph.node.append(new_node)
    
    return model

def make_node(model,slice_attr,op_type):
    for n in model.graph.node:
        if op_type in n.name:
            node = helper.make_node(
                op_type,
                name = n.name,
                inputs=[n.input[0]],
                outputs=[n.output[0]],
                **slice_attr
            )
    
            return node
    
def new_model(model):
    """
    Overwrites the main opset in an ONNX file.
    Does not change any node definition.
    :param model: ONNX model
    :param new_opset: new opset
    :return: ONNX model
    """    
    
    model_config = {}
    model_config["opset_imports"] = [onnx.helper.make_operatorsetid("",9)]
    
    graph = make_graph(
        model.graph.node, model.graph.name, model.graph.input,
        model.graph.output, model.graph.initializer)
    onnx_model = make_model(graph, functions=model.functions,**model_config)
    onnx_model.ir_version = 4 #model.ir_version
    onnx_model.producer_name = model.producer_name
    onnx_model.producer_version = model.producer_version
    onnx_model.domain = model.domain
    onnx_model.model_version = model.model_version
    onnx_model.doc_string = model.doc_string
    if len(model.metadata_props) > 0:  # pragma: no cover
        values = {p.key: p.value for p in model.metadata_props}
        set_model_props(onnx_model, values)

    return ModelWrapper(onnx_model)

def cnt_ops(model):
    nodes = []
    for n in model.graph.node:
        nodes.append(n.op_type)
    ops,cnts = np.unique(np.array(nodes),return_counts=True)
    total = {}
    for ind,k in enumerate(ops):
        total[str(k)] = cnts[ind]

    return total

In [133]:
model = ModelWrapper('tinyyolo-20210831.onnx')

model = add_attr(model, "Slice")
model = new_model(model.model)
# model = preprocess(model)

# model.save("div_add.onnx")
# model.find_consumer(model.graph.input[0].name)

model = model.transform(InferShapes())

model.save("tinyyolo-20210831_updated.onnx")
print("Saved!")


Saved!


In [134]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "tinyyolo-20210831_updated"
onnx_model = "%s.onnx" %model_file
print(onnx_model)
final_output_dir = "build_output_+%s"%model_file

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "Pynq-Z1",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

tinyyolo-20210831_updated.onnx


In [135]:
%%time
build.build_dataflow_cfg(onnx_model, cfg)

Building dataflow accelerator from tinyyolo-20210831_updated.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in build_output_+tinyyolo-20210831_updated
Build log is at build_output_+tinyyolo-20210831_updated/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Running step: step_streamline [3/17]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 264, in step_streamline
    model = model.transform(Streamline())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/streamline/__init__.py", line 98, in apply
    model = model.transform(trn)
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/batchnorm_to_affine.py", line 73, in apply
    assert len(data_shape) >= A.ndim, "Unexpected number of dims found in BatchNormToAffine"
TypeError: object of type 'NoneType' has no len()


> /workspace/qonnx/src/qonnx/transformation/batchnorm_to_affine.py(73)apply()
     71                 data_shape = model.get_tensor_shape(bn_input)
     72                 assert A.ndim == B.ndim, "Unexpected mul/add dims in BatchNormToAffine"
---> 73                 assert len(data_shape) >= A.ndim, "Unexpected number of dims found in BatchNormToAffine"
     74                 # reshape the mul/add constants to match the data shape/dims
     75                 # by adding (1,) dimensions to the right

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
CPU times: user 11.2 s, sys: 523 ms, total: 11.7 s
Wall time: 37.3 s


-1